In [2]:
import os
data_dir = os.path.expanduser("~/Google Drive/Shared drives/Data")
dataset = "bioasq"
data_path = f'{data_dir}/BEIR/{dataset}'

In [3]:
from vectorspace.utils import json_load
from beir.datasets.data_loader import GenericDataLoader
_, queries, qrels = GenericDataLoader(data_path).load(split="test")
bm25_results = json_load(f"{data_dir}/BEIR/results_{dataset}_bm25.json")

/opt/homebrew/anaconda3/lib/python3.9/site-packages/beir/datasets/data_loader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
from vectorspace.store import Store as ESDict
from elasticsearch import Elasticsearch
corpus = ESDict(Elasticsearch(['http://localhost:9200']), dataset)

In [43]:
doc_ids = [doc_id for doc_ids in bm25_results.values() for doc_id in list(doc_ids)[:1000]]

In [58]:
reduced_corpus = {}
for doc_id, doc in corpus.mget(set(doc_ids), _source=['title', 'text']):
    reduced_corpus[doc_id] = {
        'title': doc['title'] or '',
        'text': doc['text'] or ''
    }

In [56]:
from vectorspace.utils import json_dump
json_dump(reduced_corpus, f"{data_path}/reduced_corpus.json")

In [33]:
for query_id, docs in bm25_results.items():
    query = queries[query_id]
    print(query)
    print()
    for doc_id in list(docs)[:5]:
        doc = corpus[doc_id]
        print(doc['title'])
        print(doc_id in qrels[query_id])
        print()
    print()